<a href="https://colab.research.google.com/github/adib0073/TimeSeries-Using-TensorFlow/blob/main/Time_Series_Forecasting_with_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Time Series Forecasting with DNN using TensorFlow

In this notebook we will use a custom Deep Neural Network which can be used for forecasting Univariate Time Series data using TensorFlow. As an example of the time series data, we will be using the [Sunspot Data from Kaggle](https://www.kaggle.com/robervalt/sunspots). The data can be easily downloaded from my GitHub project [TimeSeries-Using-TensorFlow](https://github.com/adib0073/TimeSeries-Using-TensorFlow/blob/main/Data/Sunspots.csv). Now, let's begin! 

In [1]:
import tensorflow as tf

In [2]:
# Please validate the TF version and make sure it is above 2.0. Otherwise please download the latest TF version
print(tf.__version__)

2.4.0


### Import Modules

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Downloading and loading the data

In [10]:
!wget --no-check-certificate https://raw.githubusercontent.com/adib0073/TimeSeries-Using-TensorFlow/main/Data/Sunspots.csv -O /tmp/sunspots.csv

--2020-12-23 05:21:20--  https://raw.githubusercontent.com/adib0073/TimeSeries-Using-TensorFlow/main/Data/Sunspots.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70827 (69K) [text/plain]
Saving to: ‘/tmp/sunspots.csv’

/tmp/sunspots.csv   100%[===================>]  69.17K  --.-KB/s    in 0.01s   

2020-12-23 05:21:21 (6.21 MB/s) - ‘/tmp/sunspots.csv’ saved [70827/70827]



In [15]:
# Loading the data as a pandas dataframe
df = pd.read_csv('/tmp/sunspots.csv', index_col=0)
df.head()

,Date,Monthly Mean Total Sunspot Number
0,1749-01-31,96.7
1,1749-02-28,104.3
2,1749-03-31,116.7
3,1749-04-30,92.8
4,1749-05-31,141.7
